<a href="https://colab.research.google.com/github/Gratisfo/One-media-sentiment/blob/main/sentiment_ruBert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)

@torch.no_grad()
def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

Downloading:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/499 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/943 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/679M [00:00<?, ?B/s]

Тестирование на базе данных

In [ ]:
!pip install psycopg2-binary

In [ ]:
# добавить database.ini в .gitignore
# !/usr/bin/python
# pip install psycopg2-binary
import psycopg2
from config import config
import pandas as pd


list_of_posts = []
list_of_ids = []
df_posts = pd.DataFrame()
def connect():
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # read connection parameters
        params = config()

        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)

        # create a cursor
        cur = conn.cursor()
        cur.execute("""SELECT  message_id, content FROM message limit 500""")
        for message in cur.fetchall():
          list_of_ids.append(message[0])
          list_of_posts.append(message[1])
        df_posts['post'] = list_of_posts
        df_posts['id'] = list_of_ids
        cur.close()
        return df_posts, 
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
            print('Database connection closed.')


if __name__ == '__main__':
    connect()


Connecting to the PostgreSQL database...
Database connection closed.


In [ ]:
df_posts

In [ ]:
sentiment = []
for text in df_posts.post.to_list():
  result = predict(text)
  if result == 0:
    sentiment.append('neutral')
  elif result == 1:
    sentiment.append('positive')
  else:
    sentiment.append('negative')

In [ ]:
df_posts['sentiment'] = sentiment

In [ ]:
df_posts

,post,id,sentiment
0,Акции солидарности с политиком Алексеем Наваль...,1573,neutral
1,В эти минуты журналисты «7х7» работают на улиц...,1572,neutral
2,Что происходит в регионах вечером 21 апреля: [...,1571,neutral
3,Владимир Путин 21 апреля выступил с ежегодным ...,1570,neutral
4,В 18:30 мы начнем онлайн-трансляцию «7х7» о со...,1569,neutral
...,...,...,...
495,Армянская семья 1899г. Ереван.\n\n@Armeniorum,736,neutral
496,В Босфор вошла вторая русская эскадра; русская...,735,negative
497,"Тогда Махмуд II, увидев, что Британия занята д...",734,neutral
498,"Пожары следовали один за другим, а это свидете...",733,neutral


In [ ]:
df_posts.to_csv('data.csv')